## NOTE

1. 몽고디비에서 'plbc.ContainerIoResult', 'plbc.ContainerInOut' 두 컬렉션의 전체 데이터 가져오기
2. plbcContainerIoResult는 'copionSeq'기준, plbcContainerInOut는 '_id' 기준으로 join 진행
3. join 결과 csv파일로 저장

    >>> 각 단계별로 시간 얼마나 걸리는지 체크할 것.

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import time
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor
import json

In [2]:
mongodb_URI = " 보안상 삭제 "
client = MongoClient(mongodb_URI)
db = client.plbc

In [3]:
# 데이터 로딩에 필요한 함수 정의

def get_all_data_in_chunks(collection):
    cursor = collection.find().allow_disk_use(True)  # Remove the limit parameter
    while True:
        chunk = list(cursor)
        if not chunk:
            break
        yield chunk

def process_chunk(chunk):
    return [{fieldname: entry.get(fieldname, '') for fieldname in fieldnames} for entry in chunk]

---

# 1. 'plbc.ContainerIoResult', 'plbc.ContainerInOut' 전체 데이터 가져오기

## 1-1. plbcContainerIoResult

In [5]:
collection = db.plbcContainerIoResult

sample_document = collection.find_one()
fieldnames = set(sample_document.keys()) if sample_document else set()

start_time = time.time()
chunks = get_all_data_in_chunks(collection)
data = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in futures:
        data.extend(future.result())
loading_time = time.time() - start_time
print("plbcContainerIoResult 데이터 로딩하는 데에 걸린 시간:", loading_time, "초")

plbcContainerIoResult 데이터 로딩하는 데에 걸린 시간: 572.4970271587372 초


In [11]:
# 데이터프레임 변환 시간
start_time = time.time()
plbcContainerIoResult = pd.DataFrame(data)
end_time = time.time()
changing_time = end_time - start_time
print("데이터프레임으로 변환하는 데에 걸린 시간:", changing_time, "초")

데이터프레임으로 변환하는 데에 걸린 시간: 180.06948590278625 초


In [15]:
## 데이터 로딩 + 데이터프레임 변환 시간 합계
loading_time + changing_time

752.5665130615234

#### - 동일 데이터/동일 코드 재실행

In [10]:
collection = db.plbcContainerIoResult

sample_document = collection.find_one()
fieldnames = set(sample_document.keys()) if sample_document else set()

start_time = time.time()
chunks = get_all_data_in_chunks(collection)
data_result = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in futures:
        data_result.extend(future.result())
loading_time = time.time() - start_time
print("plbcContainerIoResult 데이터 로딩하는 데에 걸린 시간:", loading_time, "초")

plbcContainerIoResult 데이터 로딩하는 데에 걸린 시간: 754.7109789848328 초


In [11]:
# 데이터프레임 변환 시간
start_time = time.time()
plbcContainerIoResult = pd.DataFrame(data_result)
end_time = time.time()
changing_time = end_time - start_time
print("데이터프레임으로 변환하는 데에 걸린 시간:", changing_time, "초")

데이터프레임으로 변환하는 데에 걸린 시간: 276.53333473205566 초


In [19]:
# 데이터 로딩 + 데이터프레임 변환 시간 합계

loading_time + changing_time

1031.2443137168884

> ### 1차 결론) 동일 데이터/동일코드를 재실행한 결과, 시간이 증가함 (752.6초 >> 1031.2초)

---

#### - 데이터 로딩 후 한 번에 데이터프레임으로 변환 >> 데이터 로딩과 동시에 데이터프레임 변환 (시간비교)

In [14]:
collection = db.plbcContainerIoResult

sample_document = collection.find_one()
fieldnames = set(sample_document.keys()) if sample_document else set()

start_time = time.time()
chunks = get_all_data_in_chunks(collection)
data = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in futures:
        data.extend(future.result())

# 데이터프레임으로 변환
plbcContainerIoResult = pd.DataFrame(data)

loading_time = time.time() - start_time
print("plbcContainerIoResult 데이터 로딩 및 변환하는 데에 걸린 시간:", loading_time, "초")

plbcContainerIoResult 데이터 로딩 및 변환하는 데에 걸린 시간: 1082.040986776352 초


> ### 2차 결론) 데이터를 불러오면서 데이터프레임으로 변환하는 것보다, 로딩이 끝난 뒤 한 번에 변환하는 것이 더 효율적

## 1-2. plbcContainerInOut

In [5]:
collection = db.plbcContainerInOut

sample_document = collection.find_one()
fieldnames = set(sample_document.keys()) if sample_document else set()

start_time = time.time()
chunks = get_all_data_in_chunks(collection)
data_inout = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in futures:
        data_inout.extend(future.result())
loading_time = time.time() - start_time
print("plbcContainerInOut 데이터 로딩하는 데에 걸린 시간:", loading_time, "초")

plbcContainerInOut 데이터 로딩하는 데에 걸린 시간: 1786.8746728897095 초


In [7]:
# 데이터프레임 변환 시간
start_time = time.time()
plbcContainerInOut = pd.DataFrame(data_inout)
end_time = time.time()
changing_time = end_time - start_time
print("데이터프레임으로 변환하는 데에 걸린 시간:", changing_time, "초")

데이터프레임으로 변환하는 데에 걸린 시간: 2508.16152882576 초


In [8]:
## 데이터 로딩 + 데이터프레임 변환 시간 합계
loading_time + changing_time

4295.036201715469

---

# 2. 데이터 JOIN

#### plbcContainerIoResult는 'copionSeq'기준, plbcContainerInOut는 '_id' 기준으로 join 진행

In [12]:
plbcContainerIoResult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10835137 entries, 0 to 10835136
Data columns (total 13 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   ctiorReusltSts   object
 1   chgId            object
 2   ctiorReusltSts4  object
 3   copionSeq        object
 4   rgstId           object
 5   rgstDem          object
 6   ctiorReusltSts2  object
 7   remark           object
 8   _id              object
 9   ctiorReusltSts3  object
 10  ctiorResultDhms  object
 11  chgDtm           object
 12  ctiorReusltSts5  object
dtypes: object(13)
memory usage: 1.0+ GB


In [13]:
plbcContainerInOut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13626405 entries, 0 to 13626404
Data columns (total 24 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ctioIoFlag         object        
 1   ctioFaultDesc      object        
 2   ctioCntrNo2FmFlag  object        
 3   ctioCarrierRefNm   object        
 4   ctioResponseFlag   object        
 5   remark             object        
 6   ctioReserved2      object        
 7   ctioCntrNo1FmFlag  object        
 8   ctioReserved1      object        
 9   rgstDem            datetime64[ns]
 10  ctioReserved3      object        
 11  rgstId             object        
 12  _id                object        
 13  ctioVehicleRefNo   object        
 14  chgId              object        
 15  ctioBkgDhms        object        
 16  carrierId          object        
 17  ctioCntr1p1Iso     object        
 18  tmnlId             object        
 19  ctioCntr1p2Iso     object        
 20  ctioCntrNo1        obj

#### [데이터정보]

1. plbc.ContainerIoResult
      - 전체 데이터 10,835,137개
      - 칼럼 13개
---
2. plbc.ContainerInOut
      - 전체 데이터 13,626,405개
      - 칼럼 24개

In [14]:
## 데이터 join

start_time = time.time()
merged_df = pd.merge(plbcContainerIoResult, plbcContainerInOut, left_on='copionSeq', right_on='_id')
end_time = time.time()

joining_time = end_time - start_time
print("데이터 join시간:", joining_time, "초")

데이터 join시간: 1143.5207648277283 초


In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10598363 entries, 0 to 10598362
Data columns (total 37 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ctiorReusltSts     object        
 1   chgId_x            object        
 2   ctiorReusltSts4    object        
 3   copionSeq          object        
 4   rgstId_x           object        
 5   rgstDem_x          object        
 6   ctiorReusltSts2    object        
 7   remark_x           object        
 8   _id_x              object        
 9   ctiorReusltSts3    object        
 10  ctiorResultDhms    object        
 11  chgDtm             object        
 12  ctiorReusltSts5    object        
 13  ctioIoFlag         object        
 14  ctioFaultDesc      object        
 15  ctioCntrNo2FmFlag  object        
 16  ctioCarrierRefNm   object        
 17  ctioResponseFlag   object        
 18  remark_y           object        
 19  ctioReserved2      object        
 20  ctioCntrNo1FmFlag  obj

#### [데이터정보]

1. merged_df
    - 데이터 총 10,598,363개
    - 칼럼 37개

---

# 3. join 결과 csv파일로 저장

In [17]:
start_time = time.time()
merged_df.to_csv('merged_df.csv', index=False)
end_time = time.time()

saving_time = end_time - start_time
print("CSV파일 저장시간:", saving_time, "초")

CSV파일 저장시간: 148.43738985061646 초


#### - 동일 데이터/동일 코드 재실행

In [22]:
start_time = time.time()
merged_df.to_csv('merged_df2.csv', index=False)
end_time = time.time()

saving_time = end_time - start_time
print("CSV파일 저장시간:", saving_time, "초")

CSV파일 저장시간: 161.05023288726807 초


---

## 04. 전체시간

In [4]:
# 초단위
752.6 + 4295.03 + 1143.5 + 148.4

6339.53

In [5]:
# 분단위
6339.5 / 60

105.65833333333333

---

# 05. 정리

1. 콜렉션 별 전체데이터 로딩 시간 비교
   - plbc.ContainerIoResult
           - 전체 데이터 10,835,137개 (칼럼 13개)
           - 데이터 로딩시간 754.7초
           - 데이터프레임 변환시간 276.5초
           - 총 752.6초
   - plbc.ContainerInOut
           - 전체 데이터 13,626,405개 (칼럼 24개)
           - 데이터 로딩시간 1786.9초
           - 데이터프레임 변환시간 2508.2초
           - 총 4295.03초
 
2. 데이터 join
   - plbcContainerIoResult는 'copionSeq'기준
   - plbcContainerInOut는 '_id' 기준
   - join 결과 데이터 총 10,598,363개 (칼럼 37개)
   - 데이터 join 시간 1143.5초
 
3. csv파일로 저장
   - CSV 파일 저장시간 148.4초

4. 전체시간
   - 두 개 컬렉션의 전체데이터 로딩 + 데이터프레임 변환 + 테이블 JOIN + CSV파일 저장
   - 총 6339.5초 (약 105분)
 
5. 기타사항
   - 동일한 코드를 재실행 시킬 때마다 시간이 느려지는 문제를 파악하여 현재 원인을 찾고있는 중
   - JSON 형태로의 저장, batch_size 조정, 병렬로 로딩 등 시간 단축을 위한 여러 가지 방법으로 실험 진행 중